In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import GrubbsTest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve
#-----
from sklearn import svm
#-----
import scipy
from scipy import signal
#-----
from tensorflow import keras
#-----
# from tslearn.shapelets import LearningShapelets

In [ ]:
from itertools import product
def get_outg_confusion_matrix_text_colors(
    cmd
):
    r"""
    Basically taken from sklearn.metrics.ConfusionMatrixDisplay.plot
    Colors are needed so my additional text matches what's provided by sklearn method.
    
    SHOULD BE CALLED AFTER sklearn.metrics.ConfusionMatrixDisplay.plot!
    """
    #-------------------------
    cm = cmd.confusion_matrix
    assert(cm.shape[0]==2)
    n_classes = cm.shape[0]
    #-------------------------
    cmap_min, cmap_max = cmd.im_.cmap(0), cmd.im_.cmap(1.0)
    thresh = (cm.max() + cm.min()) / 2.0
    #-------------------------
    colors = np.empty((2,2), dtype=object)
    for i,j in product(range(2), range(2)):
        color = cmap_max if cm[i, j] < thresh else cmap_min
        colors[i,j] = color
    #-------------------------
    return colors

In [ ]:
def draw_confusion_matrix(
    y, 
    y_pred, 
    title=None, 
    normalize=None, 
    scientific=True, 
    ax=None, 
    text_kw=dict(fontsize='xx-large'), 
    target_eq_1_name='Outage', 
    target_eq_0_name='Baseline'
):
    r"""
    Visualize confusion matrix for outages using sklearn.metrics.ConfusionMatrixDisplay
    """
    #-------------------------
    cmd = ConfusionMatrixDisplay(
        confusion_matrix(y, y_pred, normalize=normalize), 
        display_labels=[target_eq_0_name, target_eq_1_name]
    )
    #-----
    if scientific:
        cmd.plot(values_format='.3e', ax=ax, text_kw=text_kw)
    else:
        cmd.plot(values_format='', ax=ax, text_kw=text_kw)
    #-------------------------
    cmd.ax_.set_xlabel('Predicted', fontsize='xx-large')
    cmd.ax_.set_ylabel('True', fontsize='xx-large')
    cmd.ax_.set_title(title, fontsize='xx-large', fontweight='semibold')
    #ax.set_size_inches(12, 10)
    #-------------------------
    if scientific:
        txt_fmt     = '{:.4e}'
        pct_txt_fmt = txt_fmt
    else:
        txt_fmt     = '{}'
        pct_txt_fmt = '{:.4f}'
    #-----
    ax = cmd.ax_
    ax.text(1.5, 0.9, "# Entries:  {}".format(txt_fmt).format(y.shape[0]), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.8, "# {}:  {}".format(target_eq_1_name, txt_fmt).format(y.sum()), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.7, "# {}: {}".format(target_eq_0_name, txt_fmt).format(y.shape[0]-y.sum()), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.6, "% {}:  {}".format(target_eq_1_name, pct_txt_fmt).format(100*y.sum()/y.shape[0]), fontsize=14, transform=ax.transAxes)
    #-------------------------
    ax.text(1.5, 0.4, "ACCURACY:  {:.4f}".format(accuracy_score(y, y_pred)), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.3, "PRECISION:  {:.4f}".format(precision_score(y, y_pred)), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.2, "RECALL:       {:.4f}".format(recall_score(y, y_pred)), fontsize=14, transform=ax.transAxes)
    ax.text(1.5, 0.1, "F1:               {:.4f}".format(f1_score(y, y_pred)), fontsize=14, transform=ax.transAxes)
    #--------------------------------------------------
    # Include TN, FP, FN, TP labels
    # NOTES:
    #   Text stored in cmd.text_ is stored in row-major fashion
    #   Therefore, when plotting the value, the y-value corresponds to the 0th
    #     index and the x-value corresponds to the 1st index
    #
    #   Axes are defined with limits:
    #     x_lim = (-0.5, 1.5)
    #     y_lim = (1.5, -0.5)
    #   The axes are defined such that: 
    #     top-left corner     = (-0.5, -0.5)
    #     bottom-right corner = (1.5, 1.5) 
    #-----
    colors = get_outg_confusion_matrix_text_colors(cmd)
    cat_fontsize = text_kw.get('fontsize', 'xx-large')
    #-----
    cmd.ax_.text(0,   -0.25, 'TN', ha='center', va='center', color=colors[0,0], fontweight='bold', fontsize=cat_fontsize)
    cmd.ax_.text(1.0, -0.25, 'FP', ha='center', va='center', color=colors[0,1], fontweight='bold', fontsize=cat_fontsize)

    cmd.ax_.text(0,    0.75, 'FN', ha='center', va='center', color=colors[1,0], fontweight='bold', fontsize=cat_fontsize)
    cmd.ax_.text(1.0,  0.75, 'TP', ha='center', va='center', color=colors[1,1], fontweight='bold', fontsize=cat_fontsize)    
    #--------------------------------------------------
    #return ax, cmd.ax_
    return cmd

In [ ]:
def remove_ev_submeter_in_pair(
    ami_df_i, 
    pct_0_thresh_main=0.1, 
    pct_0_thresh_subm=0.6, 
    enforce_corr=True, 
    corr_thresh=0.5, 
    #remove_undetermined=True, 
    value_col='value', 
    time_col='index', 
    PN_col='aep_premise_nb', 
    SN_col='serialnumber'
):
    r"""
    Given a pair of meter connected to a single premise, this tries to determine which is the EV submeter in the pair, 
      and keeps that which is not.
    Basically, the submeter only monitors the EV, whereas the other monitors the usage of the entire premise.
    Thus, the EV signal should be 0 for a majority of the time, and the two meters should be correlated.
    
    This is a somewhat specialized function for use with engineering the EV dataset
    
    ami_df_i:
        An AMI DF containing data for a single premise which has two meters
        
    pct_0_thresh_main:
        The maximum allowed percentage of 0 values for a meter to be considered the main meter
    pct_0_thresh_subm:
        The minimum allowed percentage of 0 values for a meter to be considered the submeter
    """
    #-------------------------
    assert(ami_df_i[PN_col].nunique()==1)
    assert(pct_0_thresh_main < pct_0_thresh_subm)
    #-------------------------
    SNs_i = ami_df_i[SN_col].unique().tolist()
    assert(len(SNs_i)<=2)
    #-------------------------
    if len(SNs_i)==1:
        return ami_df_i
    #-------------------------
    ami_df_i_1 = ami_df_i[ami_df_i[SN_col]==SNs_i[0]]
    ami_df_i_2 = ami_df_i[ami_df_i[SN_col]==SNs_i[1]]
    if time_col=='index':
        ami_df_i_1 = ami_df_i_1.sort_index()
        ami_df_i_2 = ami_df_i_2.sort_index()
    else:
        ami_df_i_1 = ami_df_i_1.sort_values(by=[time_col])
        ami_df_i_2 = ami_df_i_2.sort_values(by=[time_col])
    #--------------------------------------------------
    # NOTE: I have seen some SNs which have a bunch of 0.002 values, which, for the purposes here should be treated as 0s
    #       This is why I use <0.005 instead of ==0 and >=0.005 instead of !=0 below
    #--------------------------------------------------
    pct_1 = (ami_df_i_1[value_col]<0.005).sum()/ami_df_i_1.shape[0]
    pct_2 = (ami_df_i_2[value_col]<0.005).sum()/ami_df_i_2.shape[0]
    #-------------------------
    if pct_1 <= pct_0_thresh_main:
        defn_1 = 1
    elif pct_1 >= pct_0_thresh_subm:
        defn_1 = 0
    else:
        defn_1 = -1
    #-----
    if pct_2 <= pct_0_thresh_main:
        defn_2 = 1
    elif pct_2 >= pct_0_thresh_subm:
        defn_2 = 0
    else:
        defn_2 = -1
    #-------------------------
    if defn_1+defn_2==1:
        pass_pcts=True
    else:
        pass_pcts=False
    #-------------------------
    if not pass_pcts:
        return
    
    #--------------------------------------------------
    pass_corr=True
    if enforce_corr:
        if time_col=='index':
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_index=True, right_index=True, how='outer')
        else:
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_on=time_col, right_on=time_col, how='outer')
        # For the correlation only using values which are non-zero (and aligning, meaning non-NaN)
        ami_df_i_12 = ami_df_i_12[(ami_df_i_12[f'{value_col}_x']>=0.005) & (ami_df_i_12[f'{value_col}_y']>=0.005)]
        corr_val = ami_df_i_12[f'{value_col}_x'].corr(ami_df_i_12[f'{value_col}_y'])
        if corr_val >= corr_thresh:
            pass_corr = True
        else:
            pass_corr = False
            
    #--------------------------------------------------
    if not (pass_pcts and pass_corr):
        return
    #-------------------------
    if defn_1==1:
        assert(defn_2==0) # Sanity check, not needed
        return ami_df_i[ami_df_i[SN_col]==SNs_i[0]]
    else:
        assert(defn_1==0 and defn_2==1) # Sanity check, not needed
        return ami_df_i[ami_df_i[SN_col]==SNs_i[1]]

In [ ]:
def select_pairs_w_submeter(
    ami_df_i, 
    pct_0_thresh_main=0.1, 
    pct_0_thresh_subm=0.6, 
    enforce_corr=True, 
    corr_thresh=0.5, 
    #remove_undetermined=True, 
    value_col='value', 
    time_col='index', 
    PN_col='aep_premise_nb', 
    SN_col='serialnumber'
):
    r"""
    Given a pair of meter connected to a single premise, this tries to determine which is the EV submeter in the pair, 
      and keeps that which is not.
    Basically, the submeter only monitors the EV, whereas the other monitors the usage of the entire premise.
    Thus, the EV signal should be 0 for a majority of the time, and the two meters should be correlated.
    
    This is a somewhat specialized function for use with engineering the EV dataset
    
    ami_df_i:
        An AMI DF containing data for a single premise which has two meters
        
    pct_0_thresh_main:
        The maximum allowed percentage of 0 values for a meter to be considered the main meter
    pct_0_thresh_subm:
        The minimum allowed percentage of 0 values for a meter to be considered the submeter
    """
    #-------------------------
    assert(ami_df_i[PN_col].nunique()==1)
    assert(pct_0_thresh_main < pct_0_thresh_subm)
    #-------------------------
    SNs_i = ami_df_i[SN_col].unique().tolist()
    assert(len(SNs_i)<=2)
    #-------------------------
    if len(SNs_i)==1:
        return 
    #-------------------------
    ami_df_i_1 = ami_df_i[ami_df_i[SN_col]==SNs_i[0]]
    ami_df_i_2 = ami_df_i[ami_df_i[SN_col]==SNs_i[1]]
    if time_col=='index':
        ami_df_i_1 = ami_df_i_1.sort_index()
        ami_df_i_2 = ami_df_i_2.sort_index()
    else:
        ami_df_i_1 = ami_df_i_1.sort_values(by=[time_col])
        ami_df_i_2 = ami_df_i_2.sort_values(by=[time_col])
    #--------------------------------------------------
    # NOTE: I have seen some SNs which have a bunch of 0.002 values, which, for the purposes here should be treated as 0s
    #       This is why I use <0.005 instead of ==0 and >=0.005 instead of !=0 below
    #--------------------------------------------------
    pct_1 = (ami_df_i_1[value_col]<0.005).sum()/ami_df_i_1.shape[0]
    pct_2 = (ami_df_i_2[value_col]<0.005).sum()/ami_df_i_2.shape[0]
    #-------------------------
    if pct_1 <= pct_0_thresh_main:
        defn_1 = 1
    elif pct_1 >= pct_0_thresh_subm:
        defn_1 = 0
    else:
        defn_1 = -1
    #-----
    if pct_2 <= pct_0_thresh_main:
        defn_2 = 1
    elif pct_2 >= pct_0_thresh_subm:
        defn_2 = 0
    else:
        defn_2 = -1
    #-------------------------
    if defn_1+defn_2==1:
        pass_pcts=True
    else:
        pass_pcts=False
    #-------------------------
    if not pass_pcts:
        return
    
    #--------------------------------------------------
    pass_corr=True
    if enforce_corr:
        if time_col=='index':
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_index=True, right_index=True, how='outer')
        else:
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_on=time_col, right_on=time_col, how='outer')
        # For the correlation only using values which are non-zero (and aligning, meaning non-NaN)
        ami_df_i_12 = ami_df_i_12[(ami_df_i_12[f'{value_col}_x']>=0.005) & (ami_df_i_12[f'{value_col}_y']>=0.005)]
        corr_val = ami_df_i_12[f'{value_col}_x'].corr(ami_df_i_12[f'{value_col}_y'])
        if corr_val >= corr_thresh:
            pass_corr = True
        else:
            pass_corr = False
            
    #--------------------------------------------------
    if not (pass_pcts and pass_corr):
        return
    #-------------------------
    if defn_1==1:
        assert(defn_2==0) # Sanity check, not needed
        ami_df_i_main = ami_df_i[ami_df_i[SN_col]==SNs_i[0]].copy()
        ami_df_i_subm = ami_df_i[ami_df_i[SN_col]==SNs_i[1]].copy()
    else:
        assert(defn_1==0 and defn_2==1) # Sanity check, not needed
        ami_df_i_main =  ami_df_i[ami_df_i[SN_col]==SNs_i[1]].copy()
        ami_df_i_subm =  ami_df_i[ami_df_i[SN_col]==SNs_i[0]].copy()
    #-------------------------
    if time_col=='index':
        return_ami_df_i = pd.merge(
            ami_df_i_main, 
            ami_df_i_subm[[value_col, SN_col]], 
            left_index=True, 
            right_index=True, 
            how='inner'
        )
    else:
        return_ami_df_i = pd.merge(
            ami_df_i_main, 
            ami_df_i_subm[[value_col, time_col, SN_col]], 
            left_on=time_col, 
            right_on=time_col, 
            how='inner'
        )
    #----------
    return_ami_df_i = return_ami_df_i.rename(columns={
        f'{value_col}_x': f'{value_col}_main', 
        f'{value_col}_y': f'{value_col}_subm', 
        f'{SN_col}_x': f'{SN_col}_main', 
        f'{SN_col}_y': f'{SN_col}_subm', 
    })
    #----------
    return_ami_df_i[f'{value_col}_delt'] = return_ami_df_i[f'{value_col}_main']-return_ami_df_i[f'{value_col}_subm']
    #-------------------------
    return return_ami_df_i

# SEE WEBPAGES
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/43512887#43512887
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/56451135#56451135

In [ ]:
# class real_time_peak_detection():
#     def __init__(self, array, lag, threshold, influence):
#         self.y = list(array)
#         self.length = len(self.y)
#         self.lag = lag
#         self.threshold = threshold
#         self.influence = influence
#         self.signals = [0] * len(self.y)
#         self.filteredY = np.array(self.y).tolist()
#         self.avgFilter = [0] * len(self.y)
#         self.stdFilter = [0] * len(self.y)
#         self.avgFilter[self.lag - 1] = np.mean(self.y[0:self.lag]).tolist()
#         self.stdFilter[self.lag - 1] = np.std(self.y[0:self.lag]).tolist()

#     def thresholding_algo(self, new_value):
#         self.y.append(new_value)
#         i = len(self.y) - 1
#         self.length = len(self.y)
#         if i < self.lag:
#             return 0
#         elif i == self.lag:
#             self.signals = [0] * len(self.y)
#             self.filteredY = np.array(self.y).tolist()
#             self.avgFilter = [0] * len(self.y)
#             self.stdFilter = [0] * len(self.y)
#             self.avgFilter[self.lag] = np.mean(self.y[0:self.lag]).tolist()
#             self.stdFilter[self.lag] = np.std(self.y[0:self.lag]).tolist()
#             return 0

#         self.signals += [0]
#         self.filteredY += [0]
#         self.avgFilter += [0]
#         self.stdFilter += [0]

#         if abs(self.y[i] - self.avgFilter[i - 1]) > (self.threshold * self.stdFilter[i - 1]):

#             if self.y[i] > self.avgFilter[i - 1]:
#                 self.signals[i] = 1
#             else:
#                 self.signals[i] = -1

#             self.filteredY[i] = self.influence * self.y[i] + \
#                 (1 - self.influence) * self.filteredY[i - 1]
#             self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
#             self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])
#         else:
#             self.signals[i] = 0
#             self.filteredY[i] = self.y[i]
#             self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
#             self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])

#         return self.signals[i]


def thresholding_algo_OLD(y, lag, threshold, influence):
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    for i in range(lag, len(y)):
        if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if y[i] > avgFilter[i-1]:
                signals[i] = 1
            else:
                signals[i] = -1

            filteredY[i] = influence * y[i] + (1 - influence) * filteredY[i-1]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            signals[i] = 0
            filteredY[i] = y[i]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))


def thresholding_algo(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.mean(y)),
                    stdFilter = np.asarray(np.std(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.mean(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
def thresholding_algo_median(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.median(y)),
                    stdFilter = np.asarray(np.std(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.median(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.median(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.median(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
def mad(y):
    return np.mean(np.abs(y - np.mean(y)))

def thresholding_algo_mad(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.mean(y)),
                    stdFilter = np.asarray(mad(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = mad(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.mean(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = mad(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))


def thresholding_algo_median_mad(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.median(y)),
                    stdFilter = np.asarray(mad(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.median(y[0:lag])
    stdFilter[lag - 1] = mad(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.median(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.median(filteredY[(i-lag+1):i+1])
        stdFilter[i] = mad(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
def set_signal_groups_in_df_i(
    df_i, 
    SN_col='serialnumber',
    signal_col='signals', 
    return_signal_group_col='signal_grp'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)    
    #-------------------------    
    drop_idx = False
    if df_i.index.name in df_i.columns:
        drop_idx = True
    signals_df_i = df_i.reset_index(drop=drop_idx)[df_i.reset_index(drop=drop_idx)[signal_col]==1]
    #-----
    tmp_signal_grp_col = Utilities.generate_random_string()
    signals_df_i[tmp_signal_grp_col] = np.nan
    #-----
    tmp_idx_col = Utilities.generate_random_string()
    signals_df_i[tmp_idx_col] = signals_df_i.index
    #-----
    signals_df_i[tmp_signal_grp_col] = signals_df_i[tmp_idx_col].diff().ne(1).cumsum()
    #-------------------------
    return_df = df_i.copy()
    return_df[return_signal_group_col] = np.nan
    return_df.iloc[signals_df_i.index,-1] = signals_df_i[tmp_signal_grp_col]
    #-------------------------
    return return_df

def find_signals_and_set_signal_groups_in_df_i(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber',
    signal_col='signals', 
    return_signal_group_col='signal_grp'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    rtpd_i = thresholding_algo(
        y=df_i[value_col].tolist(), 
        lag=lag,
        threshold=threshold, 
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold
    )
    df_i[signal_col] = rtpd_i['signals']    
    #-------------------------
    return_df = set_signal_groups_in_df_i(
        df_i=df_i, 
        SN_col=SN_col,
        signal_col=signal_col, 
        return_signal_group_col=return_signal_group_col
    )
    #-------------------------
    return return_df

In [ ]:
def find_signals_and_build_df_i_signals_gpd(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
    signal_col='signals'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    assert(df_i.index.name == time_col)
    df_i = df_i.sort_index()
    #-------------------------
    signal_group_col='signal_grp'
    df_i = find_signals_and_set_signal_groups_in_df_i(
        df_i=df_i, 
        lag=lag, 
        threshold=threshold,
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold, 
        value_col=value_col, 
        SN_col=SN_col,
        signal_col=signal_col, 
        return_signal_group_col=signal_group_col
    )
    #-------------------------
    df_i_signals = df_i[df_i[signal_group_col].notna()]
    df_i_signals = df_i_signals.drop(columns=[signal_col])
    df_i_signals[signal_group_col] = df_i_signals[signal_group_col].astype(int)
    df_i_signals = df_i_signals.reset_index()
    assert(time_col in df_i_signals.columns.tolist())
    #-------------------------
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min

    df_i_signals_gpd = df_i_signals.groupby([signal_group_col]).agg({
        time_col:['mean', 'std', 'min', 'max'], 
        value_col:['mean', 'std', 'max']
    })
    df_i_signals_gpd=df_i_signals_gpd.sort_values(by=[(time_col, 'mean')])
    df_i_signals_gpd[(time_col, 'max_m_min')] = df_i_signals_gpd[(time_col, 'max')] - df_i_signals_gpd[(time_col, 'min')]
    #-------------------------
    return df_i_signals_gpd

In [ ]:
def extract_features_for_SN(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
    signal_col='signals'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min
    
    df_i_signals_gpd = find_signals_and_build_df_i_signals_gpd(
        df_i=df_i, 
        lag=lag, 
        threshold=threshold,
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold, 
        value_col=value_col, 
        SN_col=SN_col, 
        time_col=time_col, 
        signal_col=signal_col
    )
    #-------------------------
    peak_max_mean     = df_i_signals_gpd[(value_col, 'max')].mean()
    peak_width_mean   = df_i_signals_gpd[(time_col, 'max_m_min')].mean()
    peak_spacing_mean = df_i_signals_gpd[(time_col, 'mean')].diff().mean()
    # Below, the date being used is of no matter, any random date works, it's simply
    #   to make pd.to_datetime happy
    if df_i_signals_gpd.shape[0]>0:
        peak_hour_mean    = pd.to_datetime(
            '2023-01-01 ' + df_i_signals_gpd[(time_col, 'mean')].dt.strftime('%H:%M:%S'), 
            format="%Y-%m-%d %H:%M:%S"
        ).mean().round('H').time().hour
    else:
        peak_hour_mean = np.nan
    #-------------------------
    features_srs = pd.Series({
        'peak_max_mean':     peak_max_mean, 
        'peak_width_mean':   peak_width_mean, 
        'peak_spacing_mean': peak_spacing_mean, 
        'peak_hour_mean':    peak_hour_mean, 
    })
    features_srs.name = df_i[SN_col].unique()[0]
    return features_srs

In [ ]:
def extract_features_for_SN_v2(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
    signal_col='signals'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    assert(df_i.index.name == time_col)
    df_i = df_i.sort_index()
    #-------------------------
    signal_group_col='signal_grp'
    df_i = find_signals_and_set_signal_groups_in_df_i(
        df_i=df_i, 
        lag=lag, 
        threshold=threshold,
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold, 
        value_col=value_col, 
        SN_col=SN_col,
        signal_col=signal_col, 
        return_signal_group_col=signal_group_col
    )
    #-------------------------
    df_i_signals = df_i[df_i[signal_group_col].notna()]
    df_i_signals = df_i_signals.drop(columns=[signal_col])
    df_i_signals[signal_group_col] = df_i_signals[signal_group_col].astype(int)
    #-----   
    drop_idx = False
    if df_i_signals.index.name in df_i_signals.columns:
        drop_idx = True
    df_i_signals = df_i_signals.reset_index(drop=drop_idx)
    assert(time_col in df_i_signals.columns.tolist())
    #-------------------------
    # Features split into:
    #   1. Using all data in peaks pooled together
    #   2. First grouping by signal group, then extracting features
    #-------------------------
    # Features (1):
    peak_mean = df_i_signals[value_col].mean()
    peak_std  = df_i_signals[value_col].std()
    #-------------------------
    # Features (2):
    #-----
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min

    df_i_signals_gpd = df_i_signals.groupby([signal_group_col]).agg({
        time_col:['mean', 'std', 'min', 'max'], 
        value_col:['mean', 'std', 'max']
    })
    df_i_signals_gpd=df_i_signals_gpd.sort_values(by=[(time_col, 'mean')])
    df_i_signals_gpd[(time_col, 'max_m_min')] = df_i_signals_gpd[(time_col, 'max')] - df_i_signals_gpd[(time_col, 'min')]
    #-------------------------
    #-------------------------
    peak_max_mean     = df_i_signals_gpd[(value_col, 'max')].mean()
    peak_max_std      = df_i_signals_gpd[(value_col, 'max')].std()
    #-----
    peak_width_mean   = df_i_signals_gpd[(time_col, 'max_m_min')].mean()
    peak_width_std    = df_i_signals_gpd[(time_col, 'max_m_min')].std()
    #-----
    peak_spacing_mean = df_i_signals_gpd[(time_col, 'mean')].diff().mean()
    peak_spacing_std  = df_i_signals_gpd[(time_col, 'mean')].diff().std()
    #-----
    # Below, the date being used is of no matter, any random date works, it's simply
    #   to make pd.to_datetime happy
    if df_i_signals_gpd.shape[0]>0:
        peak_hour_mean    = pd.to_datetime(
            '2023-01-01 ' + df_i_signals_gpd[(time_col, 'mean')].dt.strftime('%H:%M:%S'), 
            format="%Y-%m-%d %H:%M:%S"
        ).mean().round('H').time().hour
    else:
        peak_hour_mean = np.nan
    #-------------------------
    features_srs = pd.Series({
        'peak_mean':         peak_mean, 
        'peak_std':          peak_std, 
        #-----
        'peak_max_mean':     peak_max_mean, 
        'peak_max_std':      peak_max_std, 
        #-----
        'peak_width_mean':   peak_width_mean, 
        'peak_width_std':    peak_width_std, 
        #-----
        'peak_spacing_mean': peak_spacing_mean,
        'peak_spacing_std':  peak_spacing_std,
        #-----
        'peak_hour_mean':    peak_hour_mean, 
    })
    features_srs.name = df_i[SN_col].unique()[0]
    return features_srs

In [ ]:
def make_it_funky(df_i, n_entries=500, val_col='value', SN_col='serialnumber'):
    if df_i.shape[0] < n_entries:
        return None
    assert(df_i[SN_col].nunique()==1)
    vals_i = df_i[val_col].sort_index().iloc[:n_entries].tolist()
    return_i = pd.Series(vals_i)
    return_i.name = df_i[SN_col].unique()[0]
    return return_i

In [ ]:
def smooth_peaks_OLD(
    df, 
    value_col, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    r"""
    Do not use too small a value for lag!!!!!
        This can cause the std values to become very small, causing much to be identified as peaks.
        It's not a huge deal when signal_abs_threshold!=0, because this usually gives the algorithm enough
          data to work out the rolling mean and std reliably.
    """
    #-----
    y = df[value_col].tolist()
    #-----
    if len(y) <= lag:
        return np.asarray(y)
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    

    # To avoid possibility of data starting with peak, which can mess up methods (especially if
    #   lag is small), start with first value (after lag) less than the mean
    # NOTE: The mean here includes all data, so includes peaks as well, so should be larger than the
    #         smoothed mean
    i_beg = np.argmax((y[lag:] < np.mean(y))) + lag
    
    non_signal_Y = []
    for i in range(i_beg, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = y[i]
            filteredY[i] = y[i]
        else:
            # Only looking for peaks, so don't use abs(y-avg)
            if y[i] - avgFilter[i-1] > threshold * stdFilter[i-1]:
                #-------------------------
                # First, set the signal equal to the rolling average
                #-----
                # If avgFilter[i-1]==0, try np.mean(avgFilter)
                # If both==0, use raw value, y[i]
                if avgFilter[i-1]>0:
                    signals[i] = avgFilter[i-1]
                else:
                    if np.mean(avgFilter)>0:
                        signals[i] = np.mean(avgFilter)
                    else:
                        print('Cannot use any averages!')
                        signals[i] = y[i]
                #-------------------------
                # Next, set filteredY
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    tmp_idx = np.random.randint(i-lag, i, 1)[0]
                    tmp_mean = filteredY[tmp_idx]
                    tmp_std = stdFilter[tmp_idx]
                    tmp_val  = np.random.normal(tmp_mean, tmp_std, 1)
                    filteredY[i] = influence * y[i] + (1 - influence) * tmp_val
            else:
                signals[i] = y[i]
                filteredY[i] = y[i]
        #-----
        # Note: As opposed to other thresholding functions, since any peaks have been reduced to average
        #       values, all filteredY are also non_signal_Y (so can be appended here, instead of separately
        #       in if/else statements as in other thresholding functions)
        non_signal_Y.append(filteredY[i])
        #-----
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        
    #-------------------------
    # Now, go back and perform calculation for initial lag entries using the non_signal_Y
    #   for average values
    for i in range(0,i_beg):
        if y[i] < signal_abs_threshold:
            signals[i] = y[i]
        else:
            if y[i] - np.mean(non_signal_Y) > threshold * np.std(non_signal_Y):
                # Instead of using the mean of non_signal_Y, randomly draw a mean from avgFilter
                signals[i] = np.random.choice(a=avgFilter, size=1)
            else:
                signals[i] = y[i]

    return np.asarray(signals)

In [ ]:
def smooth_peaks(
    df, 
    value_col, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    r"""
    Do not use too small a value for lag!!!!!
        This can cause the std values to become very small, causing much to be identified as peaks.
        It's not a huge deal when signal_abs_threshold!=0, because this usually gives the algorithm enough
          data to work out the rolling mean and std reliably.
          
    NOTE: While dealing with the first lag entries after i_beg, we are still susceptible to peaks contaminating
            the data (causing higher means and, more importantly, higher standard deviations)
          THEREFORE, if we are within this regime, instead of using mean and standard devation, I will instead
            use the median and interquartile range (which are much more robust against outliers)
    """
    #-----
    y = df[value_col].tolist()
    #-----
    if len(y) <= lag:
        return np.asarray(y)
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    

    # To avoid possibility of data starting with peak, which can mess up methods (especially if
    #   lag is small), start with first value (after lag) less than the mean
    # NOTE: The mean here includes all data, so includes peaks as well, so should be larger than the
    #         smoothed mean
    i_beg = np.argmax((y[lag:] < np.mean(y))) + lag
    avgFilter[i_beg - 1] = np.median(y[i_beg-lag:i_beg])
    stdFilter[i_beg - 1] = scipy.stats.iqr(y[i_beg-lag:i_beg])
    
    non_signal_Y = []
    for i in range(i_beg, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = y[i]
            filteredY[i] = y[i]
        else:
            # Only looking for peaks, so don't use abs(y-avg)
            if y[i] - avgFilter[i-1] > threshold * stdFilter[i-1]:
                #-------------------------
                # First, set the signal equal to the rolling average
                #-----
                # If avgFilter[i-1]==0, try np.mean(avgFilter)
                # If both==0, use raw value, y[i]
                if avgFilter[i-1]>0:
                    signals[i] = avgFilter[i-1]
                else:
                    if np.mean(avgFilter)>0:
                        signals[i] = np.mean(avgFilter)
                    else:
                        print('Cannot use any averages!')
                        signals[i] = y[i]
                #-------------------------
                # Next, set filteredY
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    tmp_idx = np.random.randint(i-lag, i, 1)[0]
                    tmp_mean = filteredY[tmp_idx]
                    tmp_std = stdFilter[tmp_idx]
                    tmp_val  = np.random.normal(tmp_mean, tmp_std, 1)
                    filteredY[i] = influence * y[i] + (1 - influence) * tmp_val
            else:
                signals[i] = y[i]
                filteredY[i] = y[i]
        #-----
        # Note: As opposed to other thresholding functions, since any peaks have been reduced to average
        #       values, all filteredY are also non_signal_Y (so can be appended here, instead of separately
        #       in if/else statements as in other thresholding functions)
        non_signal_Y.append(filteredY[i])
        #-----
        if i-i_beg > lag:
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            avgFilter[i] = np.median(filteredY[(i-lag+1):i+1])
            stdFilter[i] = scipy.stats.iqr(filteredY[(i-lag+1):i+1])            
        
    #-------------------------
    # Now, go back and perform calculation for initial lag entries using the non_signal_Y
    #   for average values
    for i in range(0,i_beg):
        if y[i] < signal_abs_threshold:
            signals[i] = y[i]
        else:
            if y[i] - np.mean(non_signal_Y) > threshold * np.std(non_signal_Y):
                # Instead of using the mean of non_signal_Y, randomly draw a mean from avgFilter
                signals[i] = np.random.choice(a=avgFilter, size=1)
            else:
                signals[i] = y[i]

    return np.asarray(signals)

In [ ]:
def thresholding_algo_df_i(
    df_i, 
    value_col, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold, 
    smooth_rolling_window, 
    SN_col='serialnumber', 
    PN_col='aep_premise_nb', 
    time_col='starttimeperiod_local', 
    signal_col_out='signal', 
    signal_pos_col_out='signal_pos', 
    signal_binary_col_out='signal_binary', 
    signal_pos_binary_col_out='signal_pos_binary',  
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    if df_i.index.name != time_col:
        assert(time_col in df_i.columns)
        df_i = df_i.set_index(time_col, drop=True)
    assert(df_i.index.name == time_col)
    df_i = df_i.sort_index()
    #-------------------------
    # First, smooth out the peaks so the un-biased rolling mean and std can be formed
    smooth = smooth_peaks(
        df=df_i, 
        value_col=value_col, 
        lag=lag, 
        threshold=threshold, 
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold
    )
    smooth_col = f'{value_col}_smooth'
    df_i[smooth_col] = smooth
    #-------------------------
    # Generate smooth rolling mean and std
    smooth_roll_mean_col = f'{value_col}_smooth_roll_mean'
    smooth_roll_std_col  = f'{value_col}_smooth_roll_std'
    df_i[smooth_roll_mean_col] = df_i[smooth_col].rolling(smooth_rolling_window, center=True).mean()
    df_i[smooth_roll_std_col]  = df_i[smooth_col].rolling(smooth_rolling_window, center=True).std()
    #-----
    # Drop the NaN values at front and back of df_i naturally resulting from rolling window methods
    df_i = df_i.dropna(subset=[smooth_roll_mean_col, smooth_roll_std_col]).copy()
    #-------------------------
    # Generate the signal column, which is defined for each point as the number of (rolling) std deviations 
    #   away from the (rolling) mean.
    # Also generate the binary version of the signal, which is simply whether or not the number of std away from
    #   the mean is outside of the threshold.
    # Finally, build the positive versions of the above two, which set any values less than the mean equal to 0 (since
    #   we are looking for positive peaks)
    #-----
    df_i[signal_col_out] = (df_i[value_col] - df_i[smooth_roll_mean_col])/df_i[smooth_roll_std_col]

    # Probably really only want points which are above the rolling mean, since we're looking for peaks
    # For now, keep both signal and signal_pos
    df_i[signal_pos_col_out] = df_i[signal_col_out]
    df_i.loc[df_i[signal_pos_col_out]<0, signal_pos_col_out] = 0

    # Binary results are yes/no of whether peak is found
    df_i[signal_binary_col_out]     = np.abs(df_i[signal_col_out]) > threshold
    df_i[signal_pos_binary_col_out] = np.abs(df_i[signal_pos_col_out]) > threshold
    
    #-------------------------
    # Generate a couple test series for possible use
    df_i['test_final'] = df_i[value_col]
    df_i.loc[df_i[signal_binary_col_out]==0, 'test_final'] = df_i.loc[df_i[signal_binary_col_out]==0, smooth_roll_mean_col]
    #-----
    df_i['test_final2'] = df_i[value_col]
    df_i.loc[df_i[signal_binary_col_out]==0, 'test_final2'] = 0
    #-----
    df_i['test_final_signal'] = df_i[signal_col_out]
    df_i.loc[df_i[signal_binary_col_out]==0, 'test_final_signal'] = 0
    
    #-------------------------
    # To save space, don't keep all of columns in df_i, only really keep those build here (plus SN, PN, value)
    cols_to_keep = [
        SN_col, 
        PN_col, 
        value_col, 
        smooth_col, 
        smooth_roll_mean_col, 
        smooth_roll_std_col, 
        signal_col_out, 
        signal_pos_col_out, 
        signal_binary_col_out, 
        signal_pos_binary_col_out, 
        'test_final', 
        'test_final2', 
        'test_final_signal'
    ]
    return df_i[cols_to_keep]

In [ ]:
def build_peak_features_for_df_i(
    df_i, 
    signal_group_col='signal_grp', 
    value_col='value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    if time_col not in df_i.columns:
        assert(time_col==df_i.index.name)
        df_i[time_col] = df_i.index
    assert(time_col in df_i.columns)
    #-------------------------
    df_i_signals = df_i[df_i[signal_group_col].notna()].copy()
    df_i_signals[signal_group_col] = df_i_signals[signal_group_col].astype(int)
    df_i_signals = df_i_signals.reset_index(drop=True)
    #-------------------------
    # Features split into:
    #   1. Using all data in peaks pooled together
    #   2. First grouping by signal group, then extracting features
    #-------------------------
    # Features (1):
    peak_mean = df_i_signals[value_col].mean()
    peak_std  = df_i_signals[value_col].std()
    #-------------------------
    # Features (2):
    #-----
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min

    df_i_signals_gpd = df_i_signals.groupby([signal_group_col]).agg({
        time_col:['mean', 'std', 'min', 'max'], 
        value_col:['mean', 'std', 'max']
    })
    df_i_signals_gpd=df_i_signals_gpd.sort_values(by=[(time_col, 'mean')])
    df_i_signals_gpd[(time_col, 'max_m_min')] = df_i_signals_gpd[(time_col, 'max')] - df_i_signals_gpd[(time_col, 'min')]
    #-------------------------
    #-------------------------
    peak_max_mean     = df_i_signals_gpd[(value_col, 'max')].mean()
    peak_max_std      = df_i_signals_gpd[(value_col, 'max')].std()
    #-----
    peak_width_mean   = df_i_signals_gpd[(time_col, 'max_m_min')].mean()
    peak_width_std    = df_i_signals_gpd[(time_col, 'max_m_min')].std()
    #-----
    peak_spacing_mean = df_i_signals_gpd[(time_col, 'mean')].diff().mean()
    peak_spacing_std  = df_i_signals_gpd[(time_col, 'mean')].diff().std()
    #-----
    # Below, the date being used is of no matter, any random date works, it's simply
    #   to make pd.to_datetime happy
    if df_i_signals_gpd.shape[0]>0:
        peak_hour_mean    = pd.to_datetime(
            '2023-01-01 ' + df_i_signals_gpd[(time_col, 'mean')].dt.strftime('%H:%M:%S'), 
            format="%Y-%m-%d %H:%M:%S"
        ).mean().round('H').time().hour
    else:
        peak_hour_mean = np.nan
    #-------------------------
    features_srs = pd.Series({
        'peak_mean':         peak_mean, 
        'peak_std':          peak_std, 
        #-----
        'peak_max_mean':     peak_max_mean, 
        'peak_max_std':      peak_max_std, 
        #-----
        'peak_width_mean':   peak_width_mean, 
        'peak_width_std':    peak_width_std, 
        #-----
        'peak_spacing_mean': peak_spacing_mean,
        'peak_spacing_std':  peak_spacing_std,
        #-----
        'peak_hour_mean':    peak_hour_mean, 
    })
    features_srs.name = df_i[SN_col].unique()[0]
    return features_srs

In [ ]:
fig_num=0

save_figs=True
figs_save_dir = r'C:\Users\s346557\Documents\Presentations\EVs\Figures\Samples_EVs'

In [ ]:
build_ami_df = False
files_dir = r'C:\Users\s346557\Documents\LocalData\EVs\Data'
pkl_save_dir =  r'C:\Users\s346557\Documents\LocalData\EVs'

# files_dir = r'C:\Users\s346557\Documents\LocalData\EVs\Recent_Data\Data'
# pkl_save_dir =  r'C:\Users\s346557\Documents\LocalData\EVs\Recent_Data'

In [ ]:
if build_ami_df:
    ami_df = GenAn.read_df_from_csv_dir_batches(
        files_dir=files_dir, 
        file_path_glob=r'*.csv'
    )
    #-------------------------
    ami_df = ami_df[
        (ami_df['aep_derived_uom']=='KWH') & 
        (ami_df['aep_srvc_qlty_idntfr']=='TOTAL')
    ].copy()
    #-----
    ami_df['timezoneoffset'] = ami_df['starttimeperiod'].str[-6:]
    #-------------------------
    ami_df = AMINonVee.perform_std_initiation_and_cleaning(
        ami_df, 
        timestamp_col=None
    )
    #-----
    ami_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df, 
        time_col='starttimeperiod', 
        placement_col='starttimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ami_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df, 
        time_col='endtimeperiod', 
        placement_col='endtimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ami_df=ami_df.set_index('starttimeperiod_local', drop=False)
    #-------------------------
    ami_df.to_pickle(os.path.join(pkl_save_dir, 'ami_df.pkl'))
else:
    ami_df = pd.read_pickle(os.path.join(pkl_save_dir, 'ami_df.pkl'))
#-------------------------
all_trff_dfs = pd.read_pickle(os.path.join(pkl_save_dir, 'all_trff_dfs.pkl'))

In [ ]:
ami_df['aep_premise_nb'].nunique()

In [ ]:
all_trff_dfs['PREM_NB'].nunique()

In [ ]:
all_trff_dfs_found     = all_trff_dfs[all_trff_dfs['PREM_NB'].isin(ami_df['aep_premise_nb'].unique().tolist())]
all_trff_dfs_not_found = all_trff_dfs[~all_trff_dfs['PREM_NB'].isin(ami_df['aep_premise_nb'].unique().tolist())]

In [ ]:
print(all_trff_dfs_found['PREM_NB'].nunique())
print(all_trff_dfs_not_found['PREM_NB'].nunique())

In [ ]:
print(ami_df['aep_premise_nb'].nunique())
print(ami_df['serialnumber'].nunique())

In [ ]:
# ami_df = ami_df.groupby('aep_premise_nb', as_index=False, group_keys=False).apply(
#     lambda x: remove_ev_submeter_in_pair(x)
# )

In [ ]:
print(ami_df['aep_premise_nb'].nunique())
print(ami_df['serialnumber'].nunique())

In [ ]:
ami_df_resamples = AMINonVee.build_time_resampled_dfs(
    ami_df, 
    base_freq='15T', 
    freqs=['H', '2H', '3H', '4H'], 
    other_grouper_cols=['serialnumber', 'aep_premise_nb']
)

In [ ]:
ami_df[['starttimeperiod_local', 'starttimeperiod_utc']]

In [ ]:
evs_prems = all_trff_dfs[all_trff_dfs['EV']==1]['PREM_NB'].unique().tolist()
non_prems = all_trff_dfs[all_trff_dfs['EV']==0]['PREM_NB'].unique().tolist()

# aep_premise_nb in ami_df is of type object (i.e., a string), whereas PREM_NB in trff_df is int64
evs_prems = [str(x) for x in evs_prems]
non_prems = [str(x) for x in non_prems]
#-----
ami_df_evs = ami_df[ami_df['aep_premise_nb'].isin(evs_prems)].copy()
ami_df_non = ami_df[ami_df['aep_premise_nb'].isin(non_prems)].copy()
#----
assert(ami_df.shape[0]==ami_df_evs.shape[0]+ami_df_non.shape[0])

In [ ]:
print(f'ami_df_evs.shape[0] = {ami_df_evs.shape[0]}')
print(f'ami_df_non.shape[0] = {ami_df_non.shape[0]}')

In [ ]:
print(f"ami_df_evs['aep_premise_nb'].nunique() = {ami_df_evs['aep_premise_nb'].nunique()}")
print(f"ami_df_non['aep_premise_nb'].nunique() = {ami_df_non['aep_premise_nb'].nunique()}")

In [ ]:
ami_df_resamples_evs = {}
ami_df_resamples_non = {}
for freq_i, dfs_i in ami_df_resamples.items():
    df_i = dfs_i['df']
    ami_df_evs_i = df_i[df_i['aep_premise_nb'].isin(evs_prems)].copy()
    ami_df_non_i = df_i[df_i['aep_premise_nb'].isin(non_prems)].copy()   
    #-----
    assert(df_i.shape[0]==ami_df_evs_i.shape[0]+ami_df_non_i.shape[0])
    assert(freq_i not in ami_df_resamples_evs.keys())
    assert(freq_i not in ami_df_resamples_non.keys())
    ami_df_resamples_evs[freq_i] = ami_df_evs_i
    ami_df_resamples_non[freq_i] = ami_df_non_i

# =============================================================
# =============================================================

# Simple peak finding

In [ ]:
ami_df = ami_df.sort_index()
SN = '988181190'

In [ ]:
save_figs=False

In [ ]:
df_i = ami_df[ami_df['serialnumber']==SN].sort_index().drop(columns=['starttimeperiod_local']).copy()
value_col = 'value'
freq='15T'

time_col = 'starttimeperiod_local'
lag=24
threshold=5
influence=0.0
signal_abs_threshold=1.5
n_zoom=1000

# df_i = ami_df_resamples_evs['H'][ami_df_resamples_evs['H']['serialnumber']==SN].sort_index().copy()
# value_col = 'mean_TRS value'
# freq='H'

# time_col = 'starttimeperiod_local'
# lag=12
# threshold=3
# influence=0.0
# signal_abs_threshold=1.5
# n_zoom=250

In [ ]:
#-------------------------
rtpd = thresholding_algo(
    y=df_i[value_col].tolist(), 
    lag=lag,
    threshold=threshold, 
    influence=influence, 
    signal_abs_threshold=signal_abs_threshold
)
df_i['signals'] = rtpd['signals']
#-------------------------
fig, axs = Plot_General.default_subplots(n_x=1, n_y=2)
df_i[value_col].plot(ax=axs[0], color='tab:blue')
df_i[df_i['signals']==1].reset_index().plot.scatter(
    ax=axs[0], x='starttimeperiod_local', y=value_col, color='tab:green'
)
axs[0].axhline(signal_abs_threshold, color='tab:red')
axs[0].grid(True, which='both')
# axs[0].legend().set_visible(False)
axs[0].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[0].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
df_i_zoom = df_i[:n_zoom].copy()
#-----
df_i_zoom[value_col].plot(ax=axs[1], color='tab:blue')
df_i_zoom[df_i_zoom['signals']==1].reset_index().plot.scatter(
    ax=axs[1], x='starttimeperiod_local', y=value_col, color='tab:green'
)
axs[1].axhline(signal_abs_threshold, color='tab:red')
axs[1].grid(True, which='both')
# axs[1].legend().set_visible(False)
axs[1].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[1].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
if save_figs:
    save_name = f"SimplePeakFinding_{freq}_L{lag}_T{threshold}_ST{str(signal_abs_threshold).replace('.', '_')}.png"
    Plot_General.save_fig(
        fig=fig, 
        save_dir=figs_save_dir, 
        save_name=save_name, 
        bbox_inches='tight'
    )

# Simple Peaks

In [ ]:
save_figs = False

In [ ]:
df_i = ami_df[ami_df['serialnumber']==SN].sort_index().drop(columns=['starttimeperiod_local']).copy()
value_col = 'value'
freq='15T'

time_col = 'starttimeperiod_local'
lag=24
threshold=5
influence=0.0
signal_abs_threshold=1.5
n_zoom=1000

# df_i = ami_df_resamples_evs['H'][ami_df_resamples_evs['H']['serialnumber']==SN].sort_index().copy()
# value_col = 'mean_TRS value'
# freq='H'

# time_col = 'starttimeperiod_local'
# lag=12
# threshold=3
# influence=0.0
# signal_abs_threshold=1.5
# n_zoom=250

In [ ]:
peak_df_i = extract_features_for_SN_v2(
    df_i=df_i, 
    lag=lag, 
    threshold=threshold, 
    influence=influence, 
    signal_abs_threshold=signal_abs_threshold, 
    value_col=value_col
)
#--------------------------------------------------
rtpd = thresholding_algo(
    y=df_i[value_col].tolist(), 
    lag=lag,
    threshold=threshold, 
    influence=influence, 
    signal_abs_threshold=signal_abs_threshold
)
df_i['signals'] = rtpd['signals']
#-------------------------
fig, axs = Plot_General.default_subplots(n_x=1, n_y=2)
df_i[value_col].plot(ax=axs[0], color='tab:blue')
df_i[df_i['signals']==1].reset_index().plot.scatter(
    ax=axs[0], x='starttimeperiod_local', y=value_col, color='tab:green'
)
axs[0].axhline(signal_abs_threshold, color='tab:red')
axs[0].grid(True, which='both')
# axs[0].legend().set_visible(False)
axs[0].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[0].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
df_i_zoom = df_i[:n_zoom].copy()
#-----
df_i_zoom[value_col].plot(ax=axs[1], color='tab:blue')
df_i_zoom[df_i_zoom['signals']==1].reset_index().plot.scatter(
    ax=axs[1], x='starttimeperiod_local', y=value_col, color='tab:green'
)
axs[1].axhline(signal_abs_threshold, color='tab:red')
axs[1].grid(True, which='both')
# axs[1].legend().set_visible(False)
axs[1].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[1].set_ylabel('kWh', fontsize='xx-large', loc='top')
#--------------------------------------------------
left_text_x=0.95
text_beg_y = 0.60
y_spacing = 0.035
#-------------------------
# Need to iterate over all strings first to get length of longest
# This is needed so I can align all '=' signs
max_metric_len = 0
max_value_len  = 0
for i, (metric_i, value_i) in enumerate(peak_df_i.items()):
    if isinstance(value_i, pd.Timedelta):
        value_i = value_i.round("S")
    else:
        value_i = np.round(value_i, decimals=3)
    if len(metric_i)>max_metric_len:
        max_metric_len = len(metric_i)
    if len(str(value_i))>max_value_len:
        max_value_len = len(str(value_i))
# If max_metric_len==18 and max_value_len==16, the following line will creat: '{:>18}={:<16}'
strng_fmt = f'{{:>{max_metric_len+1}}}={{:<{max_value_len+1}}}'
#-------------------------
# Now, actually print the texts
for i, (metric_i, value_i) in enumerate(peak_df_i.items()):
    if isinstance(value_i, pd.Timedelta):
        value_i = value_i.round("S")
    else:
        value_i = np.round(value_i, decimals=3)
    strng = f"{metric_i} = {value_i}"
    strng_split = strng.split('=')
    final_strng = strng_fmt.format(*strng_split)
    fig.text(
        left_text_x, 
        text_beg_y-y_spacing*i, 
        final_strng, 
        fontsize=20, 
        fontfamily='monospace', 
        ha='left'
    )
#-------------------------
if save_figs:
    save_name = f"SimplePeakMetrics_{freq}_L{lag}_T{threshold}_ST{str(signal_abs_threshold).replace('.', '_')}.png"
    Plot_General.save_fig(
        fig=fig, 
        save_dir=figs_save_dir, 
        save_name=save_name, 
        bbox_inches='tight'
    )

# New Peaks

In [ ]:
save_figs = False

In [ ]:
df_i = ami_df[ami_df['serialnumber']==SN].sort_index().copy()
value_col = 'value'
freq='15T'

time_col = 'starttimeperiod_local'
lag                   = 24
threshold             = 5
influence             = 0 
signal_abs_threshold  = 1.5
smooth_rolling_window = 48
n_zoom=1000

fit_signal_col_peaks = 'test_final'


# df_i = ami_df_resamples_evs['H'][ami_df_resamples_evs['H']['serialnumber']==SN].sort_index().copy()
# value_col = 'mean_TRS value'
# freq='H'

# time_col = 'starttimeperiod_local'
# lag                   = 12
# threshold             = 3
# influence             = 0 
# signal_abs_threshold  = 1.5
# smooth_rolling_window = 24
# n_zoom=250

# fit_signal_col_peaks = 'test_final'

In [ ]:
df_i = thresholding_algo_df_i(
    df_i=df_i, 
    value_col=value_col, 
    lag=lag, 
    threshold=threshold, 
    influence=influence, 
    signal_abs_threshold=signal_abs_threshold, 
    smooth_rolling_window=smooth_rolling_window
)
#-------------------------
peak_df_i = set_signal_groups_in_df_i(
    df_i=df_i, 
    SN_col='serialnumber',
    signal_col='signal_binary', 
    return_signal_group_col='signal_grp'     
)
#-------------------------
peak_df_i = build_peak_features_for_df_i(
    df_i=peak_df_i, 
    signal_group_col='signal_grp', 
    value_col=fit_signal_col_peaks,
    SN_col='serialnumber', 
    time_col='starttimeperiod_local',        
)
#--------------------------------------------------
fig, axs = Plot_General.default_subplots(n_x=1, n_y=2)
Plot_General.adjust_subplots_args(fig, dict(hspace=0.3))

df_i[value_col].plot(ax=axs[0], color='tab:blue', label='Raw Data')
df_i[df_i['signal_binary']==True].reset_index().plot.scatter(
    ax=axs[0], x='starttimeperiod_local', y=value_col, color='tab:green', label='Peaks'
)
df_i['test_final'].plot(ax=axs[0], color='tab:red', label='Final Data')

axs[0].legend(loc="upper right")
axs[0].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[0].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
df_i_zoom = df_i[:n_zoom].copy()
#-----
df_i_zoom[value_col].plot(ax=axs[1], color='tab:blue', label='Raw Data')
df_i_zoom[df_i_zoom['signal_binary']==True].reset_index().plot.scatter(
    ax=axs[1], x='starttimeperiod_local', y=value_col, color='tab:green', label='Peaks'
)
df_i_zoom['test_final'].plot(ax=axs[1], color='tab:red', label='Final Data')

axs[1].legend(loc="upper right")
axs[1].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[1].set_ylabel('kWh', fontsize='xx-large', loc='top')
#--------------------------------------------------
left_text_x=0.95
text_beg_y = 0.60
y_spacing = 0.035
#-------------------------
# Need to iterate over all strings first to get length of longest
# This is needed so I can align all '=' signs
max_metric_len = 0
max_value_len  = 0
for i, (metric_i, value_i) in enumerate(peak_df_i.items()):
    if isinstance(value_i, pd.Timedelta):
        value_i = value_i.round("S")
    else:
        value_i = np.round(value_i, decimals=3)
    if len(metric_i)>max_metric_len:
        max_metric_len = len(metric_i)
    if len(str(value_i))>max_value_len:
        max_value_len = len(str(value_i))
# If max_metric_len==18 and max_value_len==16, the following line will creat: '{:>18}={:<16}'
strng_fmt = f'{{:>{max_metric_len+1}}}={{:<{max_value_len+1}}}'
#-------------------------
# Now, actually print the texts
for i, (metric_i, value_i) in enumerate(peak_df_i.items()):
    if isinstance(value_i, pd.Timedelta):
        value_i = value_i.round("S")
    else:
        value_i = np.round(value_i, decimals=3)
    strng = f"{metric_i} = {value_i}"
    strng_split = strng.split('=')
    final_strng = strng_fmt.format(*strng_split)
    fig.text(
        left_text_x, 
        text_beg_y-y_spacing*i, 
        final_strng, 
        fontsize=20, 
        fontfamily='monospace', 
        ha='left'
    )
#-------------------------
if save_figs:
    save_name = f"NewPeakMetrics_{freq}_L{lag}_T{threshold}_ST{str(signal_abs_threshold).replace('.', '_')}_SRW{smooth_rolling_window}.png"
    Plot_General.save_fig(
        fig=fig, 
        save_dir=figs_save_dir, 
        save_name=save_name, 
        bbox_inches='tight'
    )

# New Keras (METHODOLOGY PLOT)

In [ ]:
save_figs=False

In [ ]:
# df_i = ami_df_resamples_evs['H'][ami_df_resamples_evs['H']['serialnumber']=='786854657'].sort_index().copy()
df_i = ami_df_resamples_evs['H'][ami_df_resamples_evs['H']['serialnumber']==SN].sort_index().copy()
value_col = 'mean_TRS value'
freq='H'

width_to_plot = '4W'

lag                   = 24
threshold             = 3
influence             = 0 
signal_abs_threshold  = 1.0
smooth_rolling_window = 48

smooth_col = f'{value_col}_smooth'
smooth_roll_mean_col = f'{value_col}_smooth_roll_mean'
smooth_roll_std_col  = f'{value_col}_smooth_roll_std'

signal_col_out='signal'
signal_pos_col_out='signal_pos'
signal_binary_col_out='signal_binary'
signal_pos_binary_col_out='signal_pos_binary'

In [ ]:
rtpd = thresholding_algo(
    y=df_i[value_col].tolist(), 
    lag=lag,
    threshold=threshold, 
    influence=influence, 
    signal_abs_threshold=signal_abs_threshold
)
df_i['simple_signals'] = rtpd['signals']
df_i_simple = df_i.copy()

In [ ]:
df_i = thresholding_algo_df_i(
    df_i=df_i, 
    value_col=value_col, 
    lag=lag, 
    threshold=threshold, 
    influence=influence, 
    signal_abs_threshold=signal_abs_threshold, 
    smooth_rolling_window=smooth_rolling_window
)

In [ ]:
og_shape = df_i.shape[0]
df_i = pd.merge(df_i, df_i_simple[['simple_signals']], left_index=True, right_index=True, how='inner')
assert(df_i.shape[0]==og_shape)

In [ ]:
df_i = df_i.loc[df_i.index[0]:df_i.index[0]+pd.Timedelta(width_to_plot)]

In [ ]:
df_i['simple_signals'].astype(int).equals(df_i['signal_binary'])

In [ ]:
df_i[df_i['simple_signals'].astype(int)!=df_i['signal_binary']]

In [ ]:
fig, axs = Plot_General.default_subplots(n_x=1, n_y=2)
Plot_General.adjust_subplots_args(fig, dict(hspace=0.3))

df_i[value_col].plot(ax=axs[0], color='tab:blue', label='Raw Data')
df_i[df_i['simple_signals']==1].reset_index().plot.scatter(
    ax=axs[0], x='starttimeperiod_local', y=value_col, color='tab:green', label='1. Peaks (1)'
)
df_i[smooth_col].plot(ax=axs[0], color='tab:pink', label='2. Peak Removed')
df_i[smooth_roll_mean_col].plot(ax=axs[0], color='tab:orange', label='3. Smoothed Peak Removed')
# axs[0].legend()
axs[0].legend(loc="upper right")
axs[0].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[0].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
df_i[value_col].plot(ax=axs[1], color='tab:blue', label='Raw Data')
df_i[df_i['signal_binary']==True].reset_index().plot.scatter(
    ax=axs[1], x='starttimeperiod_local', y=value_col, color='tab:green', label='Peaks (2)'
)
df_i['test_final'].plot(ax=axs[1], color='tab:red', label='4. Final Data')
# axs[1].legend()
axs[1].legend(loc="upper right")
axs[1].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[1].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
if save_figs:
    save_name = f"Smoothing_ex_{freq}_L{lag}_T{threshold}_ST{str(signal_abs_threshold).replace('.', '_')}_SRW{smooth_rolling_window}.png"
    Plot_General.save_fig(
        fig=fig, 
        save_dir=figs_save_dir, 
        save_name=save_name, 
        bbox_inches='tight'
    )

# New Keras

In [ ]:
save_figs=False

In [ ]:
# df_i = ami_df_resamples_evs['H'][ami_df_resamples_evs['H']['serialnumber']=='786854657'].sort_index().copy()
df_i = ami_df_resamples_evs['H'][ami_df_resamples_evs['H']['serialnumber']==SN].sort_index().copy()
value_col = 'mean_TRS value'
freq='H'

width_to_plot = '4W'

lag                   = 24
threshold             = 3
influence             = 0 
signal_abs_threshold  = 1.0
smooth_rolling_window = 48

smooth_col = f'{value_col}_smooth'
smooth_roll_mean_col = f'{value_col}_smooth_roll_mean'
smooth_roll_std_col  = f'{value_col}_smooth_roll_std'

signal_col_out='signal'
signal_pos_col_out='signal_pos'
signal_binary_col_out='signal_binary'
signal_pos_binary_col_out='signal_pos_binary'

In [ ]:
df_i = thresholding_algo_df_i(
    df_i=df_i, 
    value_col=value_col, 
    lag=lag, 
    threshold=threshold, 
    influence=influence, 
    signal_abs_threshold=signal_abs_threshold, 
    smooth_rolling_window=smooth_rolling_window
)

In [ ]:
df_i = df_i.loc[df_i.index[0]:df_i.index[0]+pd.Timedelta(width_to_plot)]

In [ ]:
fig, axs = Plot_General.default_subplots(n_x=1, n_y=2)
Plot_General.adjust_subplots_args(fig, dict(hspace=0.3))

df_i[value_col].plot(ax=axs[0], color='tab:blue', label='Raw Data')
df_i[df_i['signal_binary']==True].reset_index().plot.scatter(
    ax=axs[0], x='starttimeperiod_local', y=value_col, color='tab:green', label='1. Peaks'
)
df_i[smooth_col].plot(ax=axs[0], color='tab:pink', label='2. Peak Removed')
df_i[smooth_roll_mean_col].plot(ax=axs[0], color='tab:orange', label='3. Smoothed Peak Removed')
# axs[0].legend()
axs[0].legend(loc="upper right")
axs[0].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[0].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
df_i[value_col].plot(ax=axs[1], color='tab:blue', label='Raw Data')
df_i[df_i['signal_binary']==True].reset_index().plot.scatter(
    ax=axs[1], x='starttimeperiod_local', y=value_col, color='tab:green', label='Peaks'
)
df_i['test_final'].plot(ax=axs[1], color='tab:red', label='4. Final Data')
df_i['test_final2'].plot(ax=axs[1], color='tab:cyan', label='4b. Only Peak Kept', alpha=0.75, linestyle='dashed')
# axs[1].legend()
axs[1].legend(loc="upper right")
axs[1].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[1].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
fig.suptitle('CNN Input: kWh Values', y=0.905, fontsize='xx-large', fontweight='bold')
#-------------------------
if save_figs:
    save_name = f"CNN_values_{freq}_L{lag}_T{threshold}_ST{str(signal_abs_threshold).replace('.', '_')}_SRW{smooth_rolling_window}.png"
    Plot_General.save_fig(
        fig=fig, 
        save_dir=figs_save_dir, 
        save_name=save_name, 
        bbox_inches='tight'
    )

In [ ]:
fig, axs = Plot_General.default_subplots(n_x=1, n_y=2)
Plot_General.adjust_subplots_args(fig, dict(hspace=0.3))

df_i[value_col].plot(ax=axs[0], color='tab:blue', label='Raw Data')
df_i[df_i['signal_binary']==True].reset_index().plot.scatter(
    ax=axs[0], x='starttimeperiod_local', y=value_col, color='tab:green', label='1. Peaks'
)
df_i[smooth_col].plot(ax=axs[0], color='tab:pink', label='2. Peak Removed')
df_i[smooth_roll_mean_col].plot(ax=axs[0], color='tab:orange', label='3. Smoothed Peak Removed')
# axs[0].legend()
axs[0].legend(loc="upper right")
axs[0].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[0].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
df_i[df_i[signal_binary_col_out]==1].reset_index().plot.scatter(
    ax=axs[1], x='starttimeperiod_local', y=signal_col_out, color='tab:green', label='Peaks'
)
df_i[signal_col_out].plot(ax=axs[1], color='tab:red', label='4. Final Data')
# df_i[signal_pos_col_out].plot(ax=axs[1], color='tab:purple')
df_i['test_final_signal'].plot(ax=axs[1], color='tab:cyan', label='4b. Only Peak Kept', alpha=0.75, linestyle='dashed')
# axs[1].legend()
axs[1].legend(loc="upper right")
axs[1].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[1].set_ylabel('$n\sigma$', fontsize='xx-large', loc='top')
#-------------------------
fig.suptitle('CNN Input: kWh $\mathbf{n\sigma}$', y=0.905, fontsize='xx-large', fontweight='bold')
#-------------------------
if save_figs:
    save_name = f"CNN_nsigma_{freq}_L{lag}_T{threshold}_ST{str(signal_abs_threshold).replace('.', '_')}_SRW{smooth_rolling_window}.png"
    Plot_General.save_fig(
        fig=fig, 
        save_dir=figs_save_dir, 
        save_name=save_name, 
        bbox_inches='tight'
    )

# EV submeter idea

In [ ]:
save_figs=False

In [ ]:
ami_df_w_subm = ami_df_evs.groupby('aep_premise_nb', as_index=False, group_keys=False).apply(
    lambda x: select_pairs_w_submeter(
        ami_df_i=x, 
        pct_0_thresh_main=0.1, 
        pct_0_thresh_subm=0.6, 
        enforce_corr=True, 
        corr_thresh=0.5, 
        value_col='value', 
        time_col='index', 
        PN_col='aep_premise_nb', 
        SN_col='serialnumber'
    )
)

In [ ]:
ami_df_w_subm = ami_df_w_subm.sort_index()
ami_df_w_subm['date'] = ami_df_w_subm.index
#-------------------------
ami_df_main = ami_df_w_subm[['aep_premise_nb', 'serialnumber_main', 'value_main', 'date']].copy()
ami_df_delt = ami_df_w_subm[['aep_premise_nb', 'serialnumber_main', 'value_delt', 'date']].copy()
ami_df_subm = ami_df_w_subm[['aep_premise_nb', 'serialnumber_subm', 'value_subm', 'date']].copy()
#-------------------------
ami_df_main = ami_df_main.rename(columns={
    'serialnumber_main':'serialnumber', 
    'value_main':'value'
})

ami_df_delt = ami_df_delt.rename(columns={
    'serialnumber_main':'serialnumber', 
    'value_delt':'value'
})

ami_df_subm = ami_df_subm.rename(columns={
    'serialnumber_subm':'serialnumber', 
    'value_subm':'value'
})

In [ ]:
ami_df_w_subm['aep_premise_nb'].value_counts()

In [ ]:
n_zoom = 3000
PN = '029470841'

ami_df_main_i = ami_df_main[ami_df_main['aep_premise_nb']==PN].iloc[:n_zoom]
ami_df_delt_i = ami_df_delt[ami_df_main['aep_premise_nb']==PN].iloc[:n_zoom]
ami_df_subm_i = ami_df_subm[ami_df_main['aep_premise_nb']==PN].iloc[:n_zoom]

In [ ]:
fig, axs = Plot_General.default_subplots(n_x=1, n_y=2)
Plot_General.adjust_subplots_args(fig, dict(hspace=0.3))
ami_df_main_i['value'].plot(ax=axs[0], color='tab:blue', label='Full Data')
ami_df_subm_i['value'].plot(ax=axs[0], color='tab:green', label='EV Submeter')
axs[0].grid(True, which='both')
axs[0].legend(loc="upper right")
axs[0].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[0].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
ami_df_main_i['value'].plot(ax=axs[1], color='tab:blue', label='Full Data (target=1)')
ami_df_delt_i['value'].plot(ax=axs[1], color='tab:red', label='Full$-$EV Submeter (target=0)', alpha=0.5, linestyle='dashed')
axs[1].grid(True, which='both')
axs[1].legend(loc="upper right")
axs[1].set_xlabel('Datetime', fontsize='xx-large', loc='right')
axs[1].set_ylabel('kWh', fontsize='xx-large', loc='top')
#-------------------------
if save_figs:
    Plot_General.save_fig(
        fig=fig, 
        save_dir=figs_save_dir, 
        save_name='Using_submeters.png', 
        bbox_inches='tight'
    )